# TABLE OF CONTENTS:
---
* [Notebook Summary](#Notebook-Summary)
* [Setup](#Setup)
    * [Notebook Parameters](#Notebook-Parameters)
    * [Connect to Workspace](#Connect-to-Workspace)
* [Compute Target](#Compute-Target)
* [Pipeline Run Configuration & Environment](#Pipeline-Run-Configuration-&-Environment)
* [Pipeline Inputs](#Pipeline-Inputs)
* [Create Pipeline](#Create-Pipeline)
    * [Training Step](#Training-Step)
    * [Evaluate Step](#Evaluate-Step)
    * [Register Step](#Register-Step)
* [Publish Pipeline](#Publish-Pipeline)
* [Run Pipeline](#Run-Pipeline)
* [Resource Clean Up](#Resource-Clean-Up)
---

# Notebook Summary

In this notebook, an Azure Machine Learning (AML) training / retraining pipeline will be built and published. After building and publishing the pipeline, a REST endpoint can be used to trigger the pipeline from any HTTP library on any platform. This pipeline will be used in the MlOps process for continuous model retraining, e.g. when data or model drift is detected or in general when the model should be retrained. A pipeline gives a more operationalizable way of training than a script run (which was used for original model training in the `02_model_training` notebook) as it can be easily automated and run based on triggers. It also allows for chaining of different steps that can then be executed sequentially. In general, machine learning pipelines help to optimize the workflow in terms of speed, portability and reuse.

The training / retraining pipeline built in this notebook will consist of three different steps that are executed sequentially:
- Model training using the same code for training as in the `02_model_training` notebook
- Model evaluation (comparing the newly trained model with the model currently in production or with a manual threshold)
- Model registration (registering the newly trained model to the AML workspace based on the outcomes of the model evaluation)

Check out the [AML Documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-machine-learning-pipelines) for more info on how to build pipelines in general.

# Setup

In [17]:
# Import libraries
# import azureml.core
# import pandas as pd
from azureml.core import Dataset, Datastore, Environment, Experiment, Workspace
# from azureml.core.authentication import ServicePrincipalAuthentication
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
# from azureml.core.dataset import Dataset
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import Pipeline, PipelineData, PublishedPipeline
from azureml.pipeline.core.graph import PipelineParameter
# from azureml.pipeline.core.run import PipelineRun
from azureml.pipeline.steps import PythonScriptStep
# from datetime import datetime
# from dotenv import load_dotenv
# from pathlib import Path

print("SDK version:", azureml.core.VERSION)

SDK version: 1.14.0


### Notebook Parameters

In [18]:
# Define the name of the remote compute target cluster
cluster_name = "msai-ds3v2-03"

# Define the name of the training environment created in the 00_environment_setup notebook
env_name = "pytorch-aml-env"

# Determine whether the pipeline training run should be evaluated before model registration
run_evaluation = True

# Define the pipeline endpoint name
pipeline_name = "fowl classification model training pipeline"

# Define the pipeline endpoint version
pipeline_version = "9.0"

# Define the model_name
model_name = "fowl-model"

# Define the experiment name
experiment_name = "fowl-pytorch-training-pipeline"

### Connect to Workspace

In order to connect and communicate with the AML workspace, a workspace object needs to be instantiated using the AML SDK.

In [19]:
# Connect to the AML workspace
ws = Workspace.from_config()

# Compute Target

Retrieve a remote compute target to run the pipeline experiments on. The below code will first check whether a compute target with name **cluster_name** (defined in the [Notebook Parameters](#Notebook-Parameters) section) already exists and if it does, will retrieve it. Otherwise it will create a new compute cluster.

**Note**: At the moment it is not possible to create a new compute cluster so please specify the name of an existing compute cluster.

AML pipelines need to be run on a remote compute target and cannot be run locally.

In [20]:
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", # CPU
                                                           # vm_size='STANDARD_NC6', # GPU
                                                           max_nodes=4,
                                                           idle_seconds_before_scaledown=2400)
    
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# Use get_status() to get a detailed status for the current cluster
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-02T17:31:25.801000+00:00', 'errors': None, 'creationTime': '2020-11-06T11:23:29.070228+00:00', 'modifiedTime': '2021-03-02T13:00:12.810759+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 8, 'nodeIdleTimeBeforeScaleDown': 'PT600S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


# Pipeline Run Configuration & Environment

Load the model training environment that has been registered as part of the `00_environment_setup` notebook and use it for the pipeline run.

In [21]:
env = Environment.get(workspace=ws, name=env_name)

Create a pipeline run configuration containing the retrieved environment.

In [22]:
run_config = RunConfiguration()
run_config.environment = env

# Pipeline Inputs

Create a PipelineData object to pass data between steps.

While here the pipeline will consist of a single step only, a usual flow with multiple steps will include:
- Using Dataset objects as inputs to fetch raw data, performing some transformations, then outputting a PipelineData object.
- Use the previous step's PipelineData output object as an input object, repeated for subsequent steps.

In [23]:
pipeline_data = PipelineData("pipeline_data", datastore=ws.get_default_datastore())

Create PipelineParameter objects to be able to pass versatile arguments to the PythonScriptSteps.

In [24]:
dataset_name_param = PipelineParameter(name="dataset_name", default_value="fowl-dataset")
dataset_version_param = PipelineParameter(name="dataset_version", default_value=1)
data_file_path_param = PipelineParameter(name="data_file_path", default_value="none")
model_name_param = PipelineParameter(name="model_name", default_value="fowl-model")
caller_run_id_param = PipelineParameter(name="caller_run_id", default_value="none")

# Create Pipeline

In order to create a pipeline, the individual steps need to be created first.

A pipeline step is an object that encapsulates everything that is needed for running a pipeline including:

- environment and dependency settings
- the compute target to run the pipeline on
- input and output data, and any custom parameters
- reference to a script or SDK-logic to run during the step

There are multiple classes that inherit from the parent class PipelineStep to assist with building a step using certain frameworks and stacks. Here, the PythonScriptStep class is used to define the step logic using the train_model.py script.

An object reference in the outputs array becomes available as an input for a subsequent pipeline step, for scenarios where there is more than one step.

For a list of all classes for different step types, see the [steps package](https://docs.microsoft.com/en-gb/python/api/azureml-pipeline-steps/azureml.pipeline.steps?view=azure-ml-py).

### Training Step

Create the pipeline training step using the PipelineParameter objects.

In [25]:
train_step = PythonScriptStep(name="Train Model",
                              script_name="pipeline/train_model.py",
                              compute_target=compute_target,
                              source_directory="../src",
                              outputs=[pipeline_data],
                              arguments=["--model_name", model_name_param,
                                         "--step_output", pipeline_data,
                                         "--dataset_version", dataset_version_param,
                                         "--data_file_path", data_file_path_param,
                                         "--caller_run_id", caller_run_id_param,
                                         "--dataset_name", dataset_name_param],
                              runconfig=run_config,
                              allow_reuse=True)

print("Training step has been created.")

Training step has been created.


### Evaluate Step

Create the pipeline evaluate step using the PipelineParameter objects.

In [26]:
evaluate_step = PythonScriptStep(name="Evaluate Model",
                                 script_name="pipeline/evaluate_model.py",
                                 compute_target=compute_target,
                                 source_directory="../src",
                                 arguments=["--model_name", model_name_param,
                                            "--allow_run_cancel", True],
                                 runconfig=run_config,
                                 allow_reuse=False)

print("Evaluate step has been created.")

Evaluate step has been created.


### Register Step

Create the pipeline register step using the PipelineParameter objects.

In [27]:
register_step = PythonScriptStep(name="Register Model ",
                                 script_name="pipeline/register_model.py",
                                 compute_target=compute_target,
                                 source_directory="../src",
                                 inputs=[pipeline_data],
                                 arguments=["--model_name", model_name_param,
                                            "--step_input", pipeline_data],
                                 runconfig=run_config,
                                 allow_reuse=False)

print("Register step has been created.")

Register step has been created.


Stitch the three pipeline steps together.

In [28]:
# Check run_evaluation flag to include or exclude evaluation step.
if run_evaluation == True:
    print("Include evaluation step before register step.")
    evaluate_step.run_after(train_step)
    register_step.run_after(evaluate_step)
    steps = [train_step, evaluate_step, register_step]
else:
    print("Exclude evaluation step and directly run register step.")
    register_step.run_after(train_step)
    steps = [train_step, register_step]

Include evaluation step before register step.


Create and validate the pipeline based on the pipeline steps.

In [29]:
train_pipeline = Pipeline(workspace=ws, steps=steps)
train_pipeline._set_experiment_name
train_pipeline.validate()

Step Train Model is ready to be created [cc8734fb]Step Evaluate Model is ready to be created [329928c0]
Step Register Model  is ready to be created [22a18248]



[]

# Publish Pipeline

Publish the pipeline to create a REST endpoint that allows to rerun the pipeline from any HTTP library on any platform. The published pipeline can also be run from the AML workspace where different metdata such as run history and duration are tracked as well. If a pipeline with the same version has already been published, retrieve the existing published pipeline instead.

In [33]:
pipelines = PublishedPipeline.list(ws)
matched_pipes = []

for p in pipelines:
    if p.name == pipeline_name:
        if p.version == pipeline_version:
            matched_pipes.append(p)

if(len(matched_pipes) == 0):
    published_pipeline = train_pipeline.publish(name=pipeline_name,
                                                description="Model training/retraining pipeline",
                                                version=pipeline_version)
    
    print(f"Published pipeline '{published_pipeline.name}' with version {published_pipeline.version}.")

else:
    published_pipeline = matched_pipes[0]
    print(f"Retrieved published pipeline with id {published_pipeline.id}.")

Retrieved published pipeline with id 7a206668-537f-4c7b-a40d-ba1296e265d2.


# Run Pipeline

The first pipeline run takes more time than subsequent runs, as all dependencies must be downloaded, a Docker image is created, and the Python environment is provisioned/created. Running it again takes significantly less time as those resources are reused. Total run time depends on the workload of your scripts and processes running in each pipeline step.

In [34]:
pipeline_parameters = {"model_name": model_name}
tags = {"run_method": "jupyter_notebook"}

# Create an AML Experiment
experiment = Experiment(workspace=ws, name=experiment_name)
    
# Submit an Experiment Run using the published pipeline and defined pipeline parameters
run = experiment.submit(published_pipeline,
                        tags=tags,
                        pipeline_parameters=pipeline_parameters)

Submitted PipelineRun d34d46ad-4dca-4b48-b11d-b2f1c37cb733
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fowl-pytorch-training-pipeline/runs/d34d46ad-4dca-4b48-b11d-b2f1c37cb733?wsid=/subscriptions/981b9e4f-5b3b-40d2-82cd-fce81234947a/resourcegroups/rg-msaiempmodeltrainingamlwsf1aplateng-dev-westeurope-01/workspaces/amls-msai-f1a-dev-westeurope-01


In [35]:
# Wait for completion of the run and show output log
run.wait_for_completion(show_output=True)

PipelineRunId: d34d46ad-4dca-4b48-b11d-b2f1c37cb733
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fowl-pytorch-training-pipeline/runs/d34d46ad-4dca-4b48-b11d-b2f1c37cb733?wsid=/subscriptions/981b9e4f-5b3b-40d2-82cd-fce81234947a/resourcegroups/rg-msaiempmodeltrainingamlwsf1aplateng-dev-westeurope-01/workspaces/amls-msai-f1a-dev-westeurope-01
PipelineRun Status: Running


StepRunId: 3333ea08-5525-46cb-b1b8-7a7b84138e1a
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fowl-pytorch-training-pipeline/runs/3333ea08-5525-46cb-b1b8-7a7b84138e1a?wsid=/subscriptions/981b9e4f-5b3b-40d2-82cd-fce81234947a/resourcegroups/rg-msaiempmodeltrainingamlwsf1aplateng-dev-westeurope-01/workspaces/amls-msai-f1a-dev-westeurope-01
StepRun( Train Model ) Status: NotStarted

Streaming azureml-logs/55_azureml-execution-tvmps_0f2c7bf95ff047673a5dcb5a4f98a06a8f3494638575ff52f1ae228ad975f8eb_d.txt
2021-03-02T19:52:01Z Starting output-watcher...
2021-03-02T19:52:01Z IsD


Streaming azureml-logs/70_driver_log.txt
2021/03/02 19:52:11 Attempt 1 of http call to http://10.155.24.102:16384/sendlogstoartifacts/info
2021/03/02 19:52:11 Attempt 1 of http call to http://10.155.24.102:16384/sendlogstoartifacts/status
[2021-03-02T19:52:13.199886] Entering context manager injector.
[context_manager_injector.py] Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError'], invocation=['pipeline/train_model.py', '--model_name', 'fowl-model', '--step_output', '/mnt/batch/tasks/shared/LS_root/jobs/amls-msai-f1a-dev-westeurope-01/azureml/3333ea08-5525-46cb-b1b8-7a7b84138e1a/mounts/workspaceblobstore/azureml/3333ea08-5525-46cb-b1b8-7a7b84138e1a/pipeline_data', '--dataset_version', '1', '--data_file_path', 'none', '--caller_run_id', 'none', '--dataset_name', 'fowl-dataset'])
Script type = None
Starting the daemon thread to refresh tokens in backgr

        1.7098e-01, 1.9771e-01, 1.2457e-01, 1.4521e-01, 1.4470e-01, 1.4629e-01,
        1.4340e-01, 2.0311e-01, 1.2510e-01, 1.5752e-01, 1.4089e-01, 1.3829e-01,
        1.9652e-01, 1.3812e-01, 1.1529e-01, 1.9680e-01, 1.2544e-01, 2.1749e-01,
        1.9252e-01, 2.5120e-01, 1.3480e-01, 1.5701e-01, 2.0480e-01, 1.4346e-01,
        1.5318e-01, 1.4128e-01, 1.1794e-01, 1.8131e-01, 1.7003e-01, 1.8452e-01,
        2.1850e-01, 8.9735e-02, 1.6206e-01, 2.3236e-01, 1.2091e-01, 1.0966e-01,
        1.6427e-01, 1.3464e-01, 1.9654e-01, 1.7889e-01, 1.9975e-01, 1.6539e-01,
        1.3144e-01, 1.8920e-01, 2.3110e-01, 1.8865e-01, 2.2917e-01, 2.1113e-01,
        2.1531e-01, 2.4291e-01, 2.1853e-01, 2.0218e-01, 1.2164e-01, 1.4848e-01,
        1.2585e-01, 2.0552e-01, 1.4651e-01, 1.6583e-01, 1.3996e-01, 1.5645e-01,
        1.4790e-01, 1.2307e-01, 1.6696e-01, 1.5718e-01, 1.3467e-01, 1.5129e-01,
        1.3018e-01, 1.5325e-01, 9.6837e-02, 1.2796e-01, 1.1902e-01, 1.8961e-01,
        1.3942e-01, 1.1655e-01, 1.0167e-

         8.4216e-02,  9.1385e-02,  7.7161e-02,  1.4578e-01,  5.7966e-02,
         1.3229e-01,  1.7851e-01,  7.0813e-02,  5.5308e-04,  1.2839e-01,
         5.9885e-02,  5.9175e-02,  8.7629e-02,  8.3036e-02,  1.5537e-02,
         5.5383e-02,  1.0594e-01,  1.3908e-01,  3.0104e-02,  2.4764e-02,
         6.5882e-02,  2.6041e-02,  2.8120e-02,  5.6252e-02,  5.2728e-02,
         7.9823e-03,  1.8860e-01,  8.6247e-02,  1.1460e-01,  8.5498e-03,
         1.5749e-01,  8.7903e-02,  6.2592e-02,  8.4772e-02,  5.0236e-02,
         6.7598e-02,  4.2942e-03,  7.7839e-02,  6.4250e-02,  8.7761e-02,
         1.2996e-01,  1.2614e-01,  1.2041e-01,  1.1259e-01,  6.2039e-02,
         1.9247e-01,  1.1982e-01,  9.6458e-02,  7.2935e-02,  1.2241e-01,
         4.0697e-02,  7.3783e-02,  7.6604e-03,  1.2140e-01,  1.1785e-01,
         1.3123e-01,  2.4597e-02,  9.1845e-02,  7.1232e-02,  1.7375e-01,
         8.6875e-02, -4.0649e-04,  6.0114e-02,  1.4257e-01,  6.0326e-02,
         5.9845e-02,  9.0980e-02,  6.1240e-02,  7.8


         [[-1.4928e-02,  9.0825e-04,  5.5933e-02],
          [-1.4374e-02, -2.3072e-02,  1.5227e-02],
          [ 4.2695e-03, -2.4041e-02, -1.0686e-04]],

         [[-2.0266e-02, -2.0602e-02,  4.1681e-03],
          [-3.7535e-03,  3.7957e-03,  3.6058e-02],
          [-1.7480e-03,  3.1887e-03,  2.2409e-02]],

         ...,

         [[ 2.7498e-02,  1.7477e-03, -2.2396e-02],
          [ 1.0113e-02, -2.0220e-02, -2.6799e-02],
          [ 4.1759e-02, -1.7388e-02, -1.2517e-02]],

         [[-5.4107e-02,  3.9372e-03, -1.0757e-02],
          [-4.4691e-02,  3.2585e-02,  3.0214e-03],
          [-4.8116e-02, -6.3692e-03, -2.0016e-02]],

         [[-5.7272e-02, -6.9247e-03,  2.9370e-02],
          [-2.9502e-02, -5.4637e-03, -5.8017e-04],
          [-2.2895e-02, -4.1119e-03,  3.9352e-03]]],


        [[[-7.3416e-03,  8.5048e-03,  9.6037e-03],
          [-7.7776e-03,  9.0582e-03,  6.3730e-03],
          [ 8.3056e-04,  1.5229e-02,  5.2789e-03]],

         [[-1.6238e-02, -1.4415e-02, -7.2253e-03],
 

...


Streaming azureml-logs/75_job_post-tvmps_0f2c7bf95ff047673a5dcb5a4f98a06a8f3494638575ff52f1ae228ad975f8eb_d.txt
[2021-03-02T20:07:55.526544] Entering job release
[2021-03-02T20:07:57.089803] Starting job release
[2021-03-02T20:07:57.094984] Logging experiment finalizing status in history service.
[2021-03-02T20:07:57.095171] job release stage : upload_datastore starting...[2021-03-02T20:07:57.095870] job release stage : start importing azureml.history._tracking in run_history_release.
[2021-03-02T20:07:57.096179] job release stage : copy_batchai_cached_logs starting...Starting the daemon thread to refresh tokens in background for process with pid = 159


[2021-03-02T20:07:57.096409] job release stage : copy_batchai_cached_logs completed...
[2021-03-02T20:07:57.096685] job release stage : execute_job_release starting...
[2021-03-02T20:07:57.098912] Entering context manager injector.
[2021-03-02T20:07:57.108171] job release stage : upload_datastore completed...
[2021-03-02T20:07:57.3337




StepRunId: edd1c28d-2ed9-427b-a587-83eefb9d4a98
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/fowl-pytorch-training-pipeline/runs/edd1c28d-2ed9-427b-a587-83eefb9d4a98?wsid=/subscriptions/981b9e4f-5b3b-40d2-82cd-fce81234947a/resourcegroups/rg-msaiempmodeltrainingamlwsf1aplateng-dev-westeurope-01/workspaces/amls-msai-f1a-dev-westeurope-01
StepRun( Register Model  ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_0f2c7bf95ff047673a5dcb5a4f98a06a8f3494638575ff52f1ae228ad975f8eb_d.txt
2021-03-02T20:08:35Z Starting output-watcher...
2021-03-02T20:08:35Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-03-02T20:08:36Z Executing 'Copy ACR Details file' on 10.155.24.102
2021-03-02T20:08:36Z Copy ACR Details file succeeded on 10.155.24.102. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_14dfe23c7b0aafc031c1d7d2bb6b963b
Digest: sha256:7ad5667ac2e81210d030fe40527e9465f15d9a1e969


Streaming azureml-logs/75_job_post-tvmps_0f2c7bf95ff047673a5dcb5a4f98a06a8f3494638575ff52f1ae228ad975f8eb_d.txt
[2021-03-02T20:09:00.448963] Entering job release
[2021-03-02T20:09:01.889690] Starting job release
[2021-03-02T20:09:01.894568] Logging experiment finalizing status in history service.
[2021-03-02T20:09:01.894795] job release stage : upload_datastore starting...Starting the daemon thread to refresh tokens in background for process with pid = 169
[2021-03-02T20:09:01.895042] job release stage : start importing azureml.history._tracking in run_history_release.
[2021-03-02T20:09:01.895783] job release stage : copy_batchai_cached_logs starting...

[2021-03-02T20:09:01.896020] job release stage : copy_batchai_cached_logs completed...
[2021-03-02T20:09:01.896092] job release stage : execute_job_release starting...
[2021-03-02T20:09:01.906018] Entering context manager injector.
[2021-03-02T20:09:02.162633] job release stage : upload_datastore completed...
[2021-03-02T20:09:02.1663



PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': 'd34d46ad-4dca-4b48-b11d-b2f1c37cb733', 'status': 'Completed', 'startTimeUtc': '2021-03-02T19:51:18.227473Z', 'endTimeUtc': '2021-03-02T20:09:26.213385Z', 'properties': {'azureml.git.repository_uri': 'https://MSAI-DevOps-Org@dev.azure.com/MSAI-DevOps-Org/FormulaOne%20Azure%20CSV%20Core%20Components/_git/F1A-Platform-DataScientist-MlOps-ProjectTemplate', 'mlflow.source.git.repoURL': 'https://MSAI-DevOps-Org@dev.azure.com/MSAI-DevOps-Org/FormulaOne%20Azure%20CSV%20Core%20Components/_git/F1A-Platform-DataScientist-MlOps-ProjectTemplate', 'azureml.git.branch': 'main', 'mlflow.source.git.branch': 'main', 'azureml.git.commit': '1a66741991e0e9b3cedf875cfcafed638416fb73', 'mlflow.source.git.commit': '1a66741991e0e9b3cedf875cfcafed638416fb73', 'azureml.git.dirty': 'False', 'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK', 'azureml.parameters': '{"model_name":"fowl-model","dataset_version":"1",

'Finished'

# Resource Clean Up

Delete the compute target.

**Note**: At the moment the compute target can and should not be deleted.

In [ ]:
# compute_target.delete()